In [1]:
import os
from pathlib import Path

# Check current directory
os.getcwd()


'd:\\OneDrive\\Desktop\\disaster_victim_detection\\research'

In [2]:
# Move to project root if notebook is inside /research
os.chdir("..")
os.getcwd()


'd:\\OneDrive\\Desktop\\disaster_victim_detection'

In [3]:


from dataclasses import dataclass
from pathlib import Path


In [4]:
@dataclass
class DataIngestionConfig:
    root_dir: Path
    source_url: str
    local_data_file: Path
    unzip_dir: Path


In [5]:
from src.victimDetector.constants import *
from src.victimDetector.utils.common import read_yaml, create_directories

In [6]:
from pathlib import Path

# params.yaml is in ROOT directory, config.yaml is in config/ folder
CONFIG_FILE_PATH = Path("config/config.yaml")
PARAMS_FILE_PATH = Path("params.yaml")  # Changed! No "config/" prefix

class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

    

        create_directories([Path(self.config.artifacts_root)])

    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

In [7]:
import zipfile
import gdown
from victimDetector import logger
from victimDetector.utils.common import get_size

In [8]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    
    def download_file(self) -> str:
        '''
        Fetch data from the url
        '''
        try: 
            dataset_url = self.config.source_URL
            zip_download_dir = self.config.local_data_file
            os.makedirs("artifacts/data_ingestion", exist_ok=True)
            logger.info(f"Downloading data from {dataset_url} into file {zip_download_dir}")

            # Use the URL directly from config with fuzzy=True
            gdown.download(dataset_url, zip_download_dir, quiet=False, fuzzy=True)

            logger.info(f"Downloaded data from {dataset_url} into file {zip_download_dir}")

        except Exception as e:
            raise e
        
    
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [22]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2026-01-15 18:28:37,302: ERROR: common: Error reading YAML file: [Errno 2] No such file or directory: 'config\\config.yaml']


FileNotFoundError: [Errno 2] No such file or directory: 'config\\config.yaml'